In [4]:
# Config Variables

# Define UCI Datasets to pull
# https://archive.ics.uci.edu/ml/datasets.php
uci_datasets = [165]

# Use all data (vs just training data) for OLS to determine co-efficient sign
ols_use_all_data = True

# Training set sizes to use (only training set sizes < 60% of total dataset size will be used)
training_set_sizes = [10, 40, 160, 610, 2560, 10240, 40960]
chain_lengths = [20000, 20000, 100000, 100000, 100000, 100000, 100000]

# Number of etas (prior standard deviation) to test between 1e-04 and 1e05 (recommend 20)
# Limits at 0 and infinity are calculated separately
# Note: In practice, sigma^2 (likelihood variance) is set to 1/eta^2 and
#       lambda (penalty) is set to 1/eta^4 
num_etas = 20

# Number of folds for cross-validation (set to 300 in 2018 paper, recommend testing with 5 to see if it works)
cv_folds = 20

# Overwrite model if it's already saved to file
overwrite_model = False

In [5]:
# Load necessary libraries and helper functions

# import helper functions from helpers/data_manipulation.py
from helpers.data_manipulation import *

# import half_ridge functions from helpers/half_ridge.py
from helpers.half_ridge import *

# import dataset class
from helpers.model import Model

# import os and pickle
import os
import pickle
 
# Pandas and numpy for data manipulation
import pandas as pd
import numpy as np
np.random.seed(42)

# Matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib.ticker import FuncFormatter
import matplotlib
matplotlib.rcParams['font.size'] = 16
matplotlib.rcParams['figure.figsize'] = (9, 9)

import seaborn as sns
import time

from IPython.core.pylabtools import figsize

# Scipy helper functions
from scipy.stats import percentileofscore
from scipy import stats

# Standard ML Models for comparison
from sklearn.linear_model import LinearRegression

# Splitting data into training/testing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error

# Distributions
import scipy

# PyMC3 for Bayesian Inference
import arviz as az
import pymc as pm

In [7]:
# Load the ICU dataset, normalize the data, and split into training, validation, and test sets

# Loop over each dataset, loaded from file and formatted
for dataset in uci_datasets:
    model = Model()
    model.import_from_file('uci_ml_datasets/ucirepo_' + str(dataset) + '.csv')
    model.format_data()
    training_set_values = model.set_up_training_values(training_set_sizes, .6)

    print(training_set_values)
    
    # Initialize the comparison results
    binary_comparisons = {}

    # loop over the training set values
    for training_set_index, training_set_value in enumerate(training_set_values):
        print(f"Training Set Index: {training_set_index}, Training Set Value: {training_set_value}")

        # Check if a file exists for the dataset in the dataset_models folder
        # if it does, skip the rest of the code and continue with the loop
        # if it doesn't, continue with the rest of the code
        if os.path.isfile(f"models/uci_repo_{dataset}/{training_set_value}.csv") and not overwrite_model:
            print(f"Model already exists for dataset {dataset} and training_set_value {training_set_value}. Skipping...")
            continue

        # Generate cross-validation indices
        cv = cv_indexing(cv_folds, model.X.shape[0], training_set_value)

        # Initialize the comparison results for this training set value
        binary_comparisons[training_set_value] = {}

        # Vary eta values from 10^-4 to 10^5 and loop over them
        etas = np.logspace(-4, 5, num=num_etas)
        etas = np.insert(etas, 0, 0)
        etas = np.append(etas, np.inf)
        for eta in etas:

            # Initialize the comparison results for this eta
            binary_comparisons[training_set_value][eta] = []

            # Crossvalidate over the folds for a specific tau
            for i in range(cv_folds):

                # Print info about this iteration
                print(f"Training Set Value: {training_set_value}, Eta: {eta}, Fold: {i} of {range(cv_folds)}")

                # Get training and test indices for the i-th fold
                train_indices = np.where(cv[i] == 1)[0]
                test_indices = np.where(cv[i] == 0)[0]
                
                # Split the data
                X_train, X_test = model.X.iloc[train_indices], model.X.iloc[test_indices]
                y_train, y_test = model.y.iloc[train_indices], model.y.iloc[test_indices]

                # Determine the signs of the coefficients using OLS
                ols_coefficients = model.determine_signs_of_coefficients(ols_use_all_data, X_train, y_train)

                # Run the half ridge regression with mcmc
                #half_ridge_trace = half_ridge(X_train, y_train, model.ols_coefficients, prior_eta=eta)
                #var_dict = {}
                #for variable in half_ridge_trace.posterior:
                #    var_dict[variable] = half_ridge_trace.posterior[variable].values.flatten()
                # Results into a dataframe
                #var_weights = pd.DataFrame(var_dict)
                # Means for all the weights
                #var_means_array = var_weights.mean(axis=0)
                # Convert the dataframe var_means_array to a dictionary with the column names as keys
                #half_ridge_coefficients = {col: var_means_array[col] for col in var_weights.columns}
                
                # Run the half ridge regression with rejection sampling
                start_time = time.time()
                half_ridge_coefficients = half_ridge_rejection_sampling(ols_coefficients, X_train, y_train, eta, chain_lengths[training_set_index])
                end_time = time.time()
                duration = end_time - start_time
                print(f"Half ridge regression rejection sampling took {duration} seconds")

                # Calculate the binary comparisons for the models
                sorted_half_ridge_coefficients = {k: half_ridge_coefficients[k] for k in sorted(half_ridge_coefficients)}
                sorted_half_ridge_coefficients['intercept'] = 0
                print('half-ridge coefficients')
                print(sorted_half_ridge_coefficients)
                binary_comparison = calculate_binary_comparison(sorted_half_ridge_coefficients, X_test, y_test)
                binary_comparisons[training_set_value][eta].append(binary_comparison)
                print('binary comparisons')
                print(binary_comparison)   

    with open('model_results/ucirepo_' + str(dataset) + '.csv', 'wb') as file:
        pickle.dump(binary_comparisons, file)


[10, 40, 160, 610]
Training Set Index: 0, Training Set Value: 10
Training Set Value: 10, Eta: 0.0, Fold: 0 of range(0, 20)
Running half-ridge regression...
{'Cement': 0.7978845608028654, 'Superplasticizer': -0.7978845608028654, 'Age': -0.7978845608028654, 'Water': -0.7978845608028654, 'Fine Aggregate': -0.7978845608028654, 'Coarse Aggregate': -0.7978845608028654, 'Blast Furnace Slag': -0.7978845608028654, 'Fly Ash': -0.7978845608028654}
Half ridge regression rejection sampling took 6.771087646484375e-05 seconds
half-ridge coefficients
{'Age': -0.7978845608028654, 'Blast Furnace Slag': -0.7978845608028654, 'Cement': 0.7978845608028654, 'Coarse Aggregate': -0.7978845608028654, 'Fine Aggregate': -0.7978845608028654, 'Fly Ash': -0.7978845608028654, 'Superplasticizer': -0.7978845608028654, 'Water': -0.7978845608028654, 'intercept': 0}
Samples
9
               estimate
785  1.7223456367845547
                estimate
851  -1.3084972943513409
-1.1707328659594298
-0.17825973679217638
binary co

/workspaces/codespaces-jupyter/helpers/half_ridge.py:149: RuntimeWarning: Mean of empty slice
  post_weights = np.nanmean(samples1, axis=0)


Running half-ridge regression...
Half ridge regression rejection sampling took 0.008699893951416016 seconds
half-ridge coefficients
{'Age': -0.15466131274664274, 'Blast Furnace Slag': -0.15564057203072723, 'Cement': 0.40782769781278083, 'Coarse Aggregate': -0.5467666859484797, 'Fine Aggregate': -0.017658261701803235, 'Fly Ash': -0.009108373732450381, 'Superplasticizer': -0.2887640210711714, 'Water': -0.0850115294502686, 'intercept': 0}
Samples
7
               estimate
989  0.5401513823743703
                estimate
1021  0.4830907023770392
-0.9474563417257837
-0.2357250084448041
binary comparisons
0.466
Training Set Value: 10, Eta: 1.8329807108324339, Fold: 2 of range(0, 20)
Running half-ridge regression...
Half ridge regression rejection sampling took 0.008844137191772461 seconds
half-ridge coefficients
{'Age': -0.2555362404048659, 'Blast Furnace Slag': -0.22836384899780396, 'Cement': 0.11920915544590335, 'Coarse Aggregate': -0.7342699287867962, 'Fine Aggregate': -0.5601994192022957

/workspaces/codespaces-jupyter/helpers/half_ridge.py:149: RuntimeWarning: Mean of empty slice
  post_weights = np.nanmean(samples1, axis=0)


binary comparisons
0.619
Training Set Value: 10, Eta: 1.8329807108324339, Fold: 5 of range(0, 20)
Running half-ridge regression...
Half ridge regression rejection sampling took 0.03201174736022949 seconds
half-ridge coefficients
{'Age': 0.5611971032666971, 'Blast Furnace Slag': 0.34772119928804257, 'Cement': 0.5609819100039461, 'Coarse Aggregate': -0.24510050341682607, 'Fine Aggregate': 0.4622999467546933, 'Fly Ash': 0.09943815514518511, 'Superplasticizer': -0.24812391262594022, 'Water': -0.724591489927357, 'intercept': 0}


...

binary comparisons
0.613
Training Set Value: 610, Eta: 33598.18286283774, Fold: 2 of range(0, 20)
Running half-ridge regression...
Half ridge regression rejection sampling took 0.03702402114868164 seconds
half-ridge coefficients
{'Age': -884780804.9133872, 'Blast Furnace Slag': -956630139.4299502, 'Cement': 891067807.2427472, 'Coarse Aggregate': -892475475.7414714, 'Fine Aggregate': -847415742.9409701, 'Fly Ash': -884885327.3538623, 'Superplasticizer': -847976929.1300771, 'Water': -856090273.053231, 'intercept': 0}
Samples
4
               estimate
910  -765549166.4410075
                estimate
261  -1009901423.5882102
-0.579319445201139
0.09470030355780373
binary comparisons
0.451
Training Set Value: 610, Eta: 33598.18286283774, Fold: 3 of range(0, 20)
Running half-ridge regression...
Half ridge regression rejection sampling took 0.03472089767456055 seconds
half-ridge coefficients
{'Age': -875990232.4865781, 'Blast Furnace Slag': -942450610.8596269, 'Cement': 864121923.7182677, 'Coa

/workspaces/codespaces-jupyter/helpers/half_ridge.py:149: RuntimeWarning: Mean of empty slice
  post_weights = np.nanmean(samples1, axis=0)


binary comparisons
0.604
Training Set Value: 610, Eta: 100000.0, Fold: 1 of range(0, 20)
Running half-ridge regression...
Half ridge regression rejection sampling took 0.055416107177734375 seconds
half-ridge coefficients
{'Age': 0.47384023405470965, 'Blast Furnace Slag': 0.47307465656844755, 'Cement': 0.6117254856010124, 'Coarse Aggregate': 0.011654031746729002, 'Fine Aggregate': 0.05800999717190959, 'Fly Ash': 0.2826295864614272, 'Superplasticizer': 0.09424416483971332, 'Water': -0.23503310374855907, 'intercept': 0}
Samples
10
               estimate
145  0.9978085596007195
                  estimate
419  -0.053119516729477345
2.0042233928482367
-1.133619878017108
binary comparisons
0.642
Training Set Value: 610, Eta: 100000.0, Fold: 2 of range(0, 20)


/workspaces/codespaces-jupyter/helpers/half_ridge.py:149: RuntimeWarning: Mean of empty slice
  post_weights = np.nanmean(samples1, axis=0)


Running half-ridge regression...
Half ridge regression rejection sampling took 0.04295086860656738 seconds
half-ridge coefficients
{'Age': -8357842928.897527, 'Blast Furnace Slag': -8052549370.205781, 'Cement': 7350802311.114987, 'Coarse Aggregate': -7987346460.752775, 'Fine Aggregate': -8050465957.747843, 'Fly Ash': -8299798834.411721, 'Superplasticizer': -8188722466.272511, 'Water': -8358239729.339645, 'intercept': 0}
Samples
8
               estimate
556  423590011.25028706
                estimate
909  -1365020631.7901964
-0.19083026496618868
-0.7229826243327078
binary comparisons
0.475
Training Set Value: 610, Eta: 100000.0, Fold: 3 of range(0, 20)
Running half-ridge regression...
Half ridge regression rejection sampling took 0.03606581687927246 seconds
half-ridge coefficients
{'Age': -7609526065.788654, 'Blast Furnace Slag': -8011412838.27299, 'Cement': 7636857263.394848, 'Coarse Aggregate': -8290252531.53367, 'Fine Aggregate': -7799970607.244406, 'Fly Ash': -7628518447.791714, '